<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_6_18%E5%9B%9E%E7%9B%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9回目の提出さ

ここまでのtry
*   Light GBM、特徴量12個くらい。MinMaxScalerで正規化したけど、不必要かも
*   基準を0.1以上をOKとしたらクリアになった。？？？？怪しいな
*   crossvalidationがちゃんと働いてないな。
*　 cataegorical featuresという形でもうできることがわかったぞ


---


今後の課題
*   optunaを入れて調整しよう
*   過学習の判断をできるようにしよう
*   Minmaxとかの正規化ってcategoricaldataにもやっていいの？
*   
*   


In [2]:
#importまとめ。pandas-profilingは少々導入が面倒
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file
%matplotlib inline

warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-23rwa48m
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-23rwa48m
     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 604kB 18.2MB/s 
     |████████████████████████████████| 3.1MB 25.2MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 296kB 36.7MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=ffc150ffced04a8c34e9af054111835a36536b842b3b924cd02854acf3067219
  Stored in directory: /tmp/pip-ephem-wheel-cache-1lrzzqt5/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=fcc6e0ffe4991d7f2b3d7170cfdad16ec9e413aabaee6fb9864c1f9d854342b

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data


In [4]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving submit_sample.csv to submit_sample.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [51]:
!ls

sample_data  submit_sample.csv	test.csv  train.csv


In [136]:
#上手く列指定ができなかったので、先頭にheaderをつける
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
#test,train,その足し合わせdataの三種を作る
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
train=train.sort_values(by='y')
"""
y1train=sum(train["y"])
train.index = range(len(train))
train=train.drop(range(len(train)-3*y1train))
train = train.sample(frac=1)
train.index = range(len(train))
"""

'\ny1train=sum(train["y"])\ntrain.index = range(len(train))\ntrain=train.drop(range(len(train)-3*y1train))\ntrain = train.sample(frac=1)\ntrain.index = range(len(train))\n'

In [137]:

data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
data["job"] = data["job"].astype(np.int64)
data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)
#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)

In [138]:
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)

In [139]:
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)

In [140]:
data

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,income,goodincome,married,single,year,campaign+previous,poutcome/pdays
0,0,31,4,married,3,no,12294,1,0,2,21,1,101,3,498,0,0,0.0,0,0,married,married,51,3,0
17687,17687,34,7,married,3,no,91274,1,0,0,15,7,152,2,54,0,2,0.0,0,1,married,married,225,2,312
17686,17686,29,3,divorced,3,no,49151,0,1,2,7,9,271,3,730,0,2,0.0,0,1,divorced,divorced,277,3,27
17685,17685,29,10,single,1,no,15822,0,0,2,21,1,158,1,194,0,2,0.0,1,0,single,single,51,1,98
17684,17684,35,2,married,1,no,5823,0,0,0,26,7,345,1,189,0,2,0.0,0,0,married,married,236,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,18045,49,6,married,1,no,98357,1,0,2,6,9,101,2,417,0,1,NaN,0,1,married,married,276,2,23
18046,18046,34,0,married,3,no,29621,1,0,2,12,7,345,1,815,0,2,NaN,0,0,married,married,222,1,24
18047,18047,34,3,single,3,no,94260,1,0,0,16,7,121,2,370,0,2,NaN,0,1,single,single,226,2,52
18048,18048,31,2,single,3,no,65483,1,0,0,15,7,345,2,41,0,2,NaN,0,0,single,single,225,2,392


In [141]:
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)

#全体のうち、残したいモノだけ選ぶ
all_columns = list(train.columns)
remain_columns = ['age','job','education','balance','housing','loan',
                  'contact','day','month','duration','campaign','pdays','previous',\
                  'poutcome','y','income','goodincome',"married","single","year",
                  'campaign+previous'	,'poutcome/pdays'	,'housing+loan','duration*campaign']
                  
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]


**------------------cross validation--------------------**


**------light GBM-------**

In [142]:
#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y_train = train['y']
X_train = train.drop('y', axis = 1)
X_test = test.drop('y', axis = 1)

"""
# MinmaxScalarを使って正規化。
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
# 訓練用のデータを正規化する。fit_transformはfit＆transformの略で、情報取得＆正規化の意味
X_train_norm = mmsc.fit_transform(X_train)
# 訓練用データを基準にテストデータも正規化
X_test_norm = mmsc.transform(X_test)
X_train=X_train_norm
X_test=X_test_norm
#正規化するとarrayになっていたので、DataFrameに戻す
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
#Dataflameにしたらheaderが抜けていたので付け直す
X_train.columns=remain_columns
X_test.columns=remain_columns
#この辺のheader周りの操作は無駄なのでできたら修正したい
"""

'\n# MinmaxScalarを使って正規化。\nfrom sklearn.preprocessing import MinMaxScaler\nmmsc = MinMaxScaler()\n# 訓練用のデータを正規化する。fit_transformはfit＆transformの略で、情報取得＆正規化の意味\nX_train_norm = mmsc.fit_transform(X_train)\n# 訓練用データを基準にテストデータも正規化\nX_test_norm = mmsc.transform(X_test)\nX_train=X_train_norm\nX_test=X_test_norm\n#正規化するとarrayになっていたので、DataFrameに戻す\nX_train=pd.DataFrame(X_train)\nX_test=pd.DataFrame(X_test)\n#Dataflameにしたらheaderが抜けていたので付け直す\nX_train.columns=remain_columns\nX_test.columns=remain_columns\n#この辺のheader周りの操作は無駄なのでできたら修正したい\n'

In [143]:
#欲しいデータが見えているか確認
data

,age,job,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,income,goodincome,married,single,year,campaign+previous,poutcome/pdays,housing+loan,duration*campaign
0,31,4,3,12294,1,0,2,21,1,101,3,498,0,0,0.0,0,0,1,1,51,3,0,1,303
17687,34,7,3,91274,1,0,0,15,7,152,2,54,0,2,0.0,0,1,1,1,225,2,312,1,304
17686,29,3,3,49151,0,1,2,7,9,271,3,730,0,2,0.0,0,1,1,0,277,3,27,1,813
17685,29,10,1,15822,0,0,2,21,1,158,1,194,0,2,0.0,1,0,0,0,51,1,98,0,158
17684,35,2,1,5823,0,0,0,26,7,345,1,189,0,2,0.0,0,0,1,1,236,1,100,0,345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,1,98357,1,0,2,6,9,101,2,417,0,1,NaN,0,1,1,1,276,2,23,1,202
18046,34,0,3,29621,1,0,2,12,7,345,1,815,0,2,NaN,0,0,1,1,222,1,24,1,345
18047,34,3,3,94260,1,0,0,16,7,121,2,370,0,2,NaN,0,1,0,0,226,2,52,1,242
18048,31,2,3,65483,1,0,0,15,7,345,2,41,0,2,NaN,0,0,0,0,225,2,392,1,690


In [144]:
remain_columns

['age',
 'job',
 'education',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y',
 'income',
 'goodincome',
 'married',
 'single',
 'year',
 'campaign+previous',
 'poutcome/pdays',
 'housing+loan',
 'duration*campaign']

In [145]:
X_train

,age,job,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,income,goodincome,married,single,year,campaign+previous,poutcome/pdays,housing+loan,duration*campaign
0,31,4,3,12294,1,0,2,21,1,101,3,498,0,0,0,0,1,1,51,3,0,1,303
17687,34,7,3,91274,1,0,0,15,7,152,2,54,0,2,0,1,1,1,225,2,312,1,304
17686,29,3,3,49151,0,1,2,7,9,271,3,730,0,2,0,1,1,0,277,3,27,1,813
17685,29,10,1,15822,0,0,2,21,1,158,1,194,0,2,1,0,0,0,51,1,98,0,158
17684,35,2,1,5823,0,0,0,26,7,345,1,189,0,2,0,0,1,1,236,1,100,0,345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15068,49,7,1,90766,0,0,1,3,6,66,1,389,0,2,0,1,1,1,183,1,50,0,66
11118,34,11,1,97710,0,0,0,30,7,658,1,819,0,2,0,0,0,0,240,1,24,0,658
15066,47,0,2,13431,0,0,0,16,7,152,3,298,0,2,0,0,1,1,226,3,64,0,456
2368,56,0,3,97717,0,0,1,2,8,177,2,556,0,2,0,0,0,0,242,2,35,0,354


In [146]:
X_train.dtypes

age                  int64
job                  int64
education            int64
balance              int64
housing              int64
loan                 int64
contact              int64
day                  int64
month                int64
duration             int64
campaign             int64
pdays                int64
previous             int64
poutcome             int64
income               int64
goodincome           int64
married              int64
single               int64
year                 int64
campaign+previous    int64
poutcome/pdays       int64
housing+loan         int64
duration*campaign    int64
dtype: object

In [ ]:
#LightGBMに入れる。
from sklearn.model_selection import KFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=3, shuffle=True, random_state=0)

#categorical dataの入れ忘れに注意
categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']

params = {
    'objective': 'binary',
    'max_bin': 1000,
    'learning_rate': 0.01,
    'num_leaves': 40,
    'max_depth' : 3
}


for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)
    
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.268136	valid_1's binary_logloss: 0.253464
[20]	training's binary_logloss: 0.261107	valid_1's binary_logloss: 0.247597
[30]	training's binary_logloss: 0.255886	valid_1's binary_logloss: 0.243318
[40]	training's binary_logloss: 0.251771	valid_1's binary_logloss: 0.239975
[50]	training's binary_logloss: 0.248433	valid_1's binary_logloss: 0.237303
[60]	training's binary_logloss: 0.245672	valid_1's binary_logloss: 0.23511
[70]	training's binary_logloss: 0.243289	valid_1's binary_logloss: 0.233236
[80]	training's binary_logloss: 0.241143	valid_1's binary_logloss: 0.231526
[90]	training's binary_logloss: 0.239171	valid_1's binary_logloss: 0.229931
[100]	training's binary_logloss: 0.237449	valid_1's binary_logloss: 0.228593
[110]	training's binary_logloss: 0.23577	valid_1's binary_logloss: 0.227266
[120]	training's binary_logloss: 0.234061	valid_1's binary_logloss: 0.225828
[130]	training's binary_l

In [ ]:
#pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score
percentage=0.035

y_pred_oof = (oof_train > percentage).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
y_sub = sum(y_preds) / len(y_preds)
fig = plt.figure(figsize=(120.0, 6.0))
plt.plot(y_sub)
y_sub = (y_sub > percentage).astype(int)


sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

In [ ]:
x = range(len(train))
y = oof_train
fig = plt.figure(figsize=(120.0, 6.0))
plt.plot(x, y)

#plt.xlim([0,10000])
plt.show()

In [ ]:
fig = plt.figure(figsize=(120.0, 6.0))
plt.plot(y_sub)


In [ ]:
sub

-----ここまでcross validation-------

In [ ]:
files.download('submission_lightgbm_kfold.csv')